In [19]:
# import os
# import re
# import pickle
# import numpy as np

# from PIL import Image

# %matplotlib inline

# from matplotlib import pyplot as plt

# from itertools import combinations
# import random
# from copy import deepcopy

# import argparse

# masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

# events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'

# new_dataset_path = os.path.dirname(os.path.basename(__file__),'train_dataset_test')

# dataset_path = os.path.join(os.getcwd(),'dataset') #Provisional path


def create_dataset_folders(new_dataset_path):
    
    for stage in ['train','val','test']:
        for files in ['images','masks']:
            os.makedirs(os.path.join(new_dataset_path,stage,files),exist_ok=True)




# seed = 42
# test_split_ratio = 0.1
# val_split_ratio = 0.1

# random.seed(seed)

# masks_paths = os.listdir(masks_path)

# random.shuffle(masks_paths)


def get_n_events_per_scene_dict(dataset_path: str,
                                seed: int = 42):

    random.seed(seed)

    masks_paths = os.listdir(os.path.join(dataset_path,'masks','event','segmentation_masks'))
    
    random.shuffle(masks_paths)

    scene_list = []

    event_scenes_dict = {}

    for mask_name in masks_paths:

        scene_name = re.match(r'(.+)_[0-9]+_G',mask_name).group(1)

        if re.match(r'(.+)_[0-9]+',scene_name): #This ensures tha scenes with multiple numbers such as Raung_1 and Raung are classified as the same scene
            scene_name = re.match(r'(.+)_[0-9]+',scene_name).group(1)
        
        if scene_name not in scene_list:
            number_of_images_per_scene = sum(1 for scene in masks_paths if scene.startswith(scene_name))
            scene_list.append(scene_name)
            event_scenes_dict[scene_name] = number_of_images_per_scene

    assert(len(masks_paths)==sum(event_scenes_dict.values())) # Ensure that all the events were correctly categorized
    
    return event_scenes_dict


def find_scenes_combination(n_elems_per_comb: int = 5,
                            input_dict: dict = None,
                            n_events_max: int = None):

    # Number of maximum iterations
    # print(int(math.factorial(len(input_dict))/(math.factorial(n_elems_per_comb)*math.factorial(len(input_dict)-n_elems_per_comb))))

    for combination in combinations(input_dict.values(), n_elems_per_comb):
        
        if sum(combination) > n_events_max-3 and sum(combination) < n_events_max+3:
            used_values = []
            scenes_combination = []
            for value in combination:
                    for key,val in input_dict.items():
                        if val == value and key not in used_values:
                            scenes_combination.append(key)
                            used_values.append(val)
                            break                    
            
            return scenes_combination


# event_scenes_dict = get_n_events_per_scene_dict(masks_paths)


# # Maximum number of events in tests
# n_events_test_max = int(test_split_ratio * sum(event_scenes_dict.values()))

# scenes_test_combination = find_scenes_combination(input_dict=event_scenes_dict,n_events_max=n_events_test_max)

# scenes_val_combination = find_scenes_combination(input_dict=events_copy_dict,n_events_max=val_max_idx) # No need to perform the same on the validation

def geo_splitted_events_creation(dataset_path: str,
                                 new_dataset_path: str,
                                 event_scenes_dict: dict,
                                 scenes_test_combination: list,
                                 val_split_ratio: float = 0.1,
                                 seed: int = 42):

    random.seed(seed)

    masks_path = os.path.join(dataset_path,'masks','event','segmentation_masks')
    events_path = os.path.join(dataset_path,'images','event','NIR_SWIR')

    masks_paths =os.listdir(masks_path)
    
    random.shuffle(masks_paths)
    
    events_copy_dict = deepcopy(event_scenes_dict)
    removed_scenes_names = [] # List used for removing the names of the scenes that have been already classified into testing or validation
    val_idx = 0
    val_max_idx = int(val_split_ratio * sum(event_scenes_dict.values()))

    for event_mask_name in masks_paths:

        patch_name = re.match(r'(.+)_mask',event_mask_name).group(1)
        patch_name = event_mask_name.replace('mask_weakly','NIR_SWIR')
        
        with open(os.path.join(masks_path,event_mask_name),'rb') as mask_file:
            mask = pickle.load(mask_file)

        mask[mask == 122] = 0
        # mask[mask == 255] = 1

        mask_img = Image.fromarray(mask)

        with open(os.path.join(events_path,patch_name),'rb') as image_file:
            image = pickle.load(image_file)
            image_uint8 = np.round(np.multiply(image,255)).astype(np.uint8)

            image_img = Image.fromarray(image_uint8)


        scene_name = re.match(r'(.+)_[0-9]+_G',event_mask_name).group(1)

        if re.match(r'(.+)_[0-9]+',scene_name): #This ensures tha scenes with multiple numbers such as Raung and Raung 1 are classified as the same
            scene_name = re.match(r'(.+)_[0-9]+',scene_name).group(1)


        if scene_name in scenes_test_combination:
            if scene_name not in removed_scenes_names:
                del events_copy_dict[scene_name]
                removed_scenes_names.append(scene_name)

            
            mask_img.save(os.path.join(os.path.join(new_dataset_path,'test','masks'),event_mask_name.replace('_mask_weakly.pkl','_mask.png')))
            image_img.save(os.path.join(os.path.join(new_dataset_path,'test','images'),patch_name.replace('.pkl','.png')))
            
        else:
            if val_idx <val_max_idx:

                val_idx +=1
        
                mask_img.save(os.path.join(os.path.join(new_dataset_path,'val','masks'),event_mask_name.replace('_mask_weakly.pkl','_mask.png')))
                image_img.save(os.path.join(os.path.join(new_dataset_path,'val','images'),patch_name.replace('.pkl','.png')))
        
            else:
                mask_img.save(os.path.join(os.path.join(new_dataset_path,'train','masks'),event_mask_name.replace('_mask_weakly.pkl','_mask.png')))
                image_img.save(os.path.join(os.path.join(new_dataset_path,'train','images'),patch_name.replace('.pkl','.png')))            

    print(f'Events generated.')

# geo_splitted_events_creation(dataset_path=dataset_path,
#                              new_dataset_path = new_dataset_path,
#                              event_scenes_dict = event_scenes_dict,
#                              scenes_test_combination = scenes_test_combination)

In [20]:
# import os
# import re
# import pickle
# import numpy as np


# from PIL import Image
# import random

# masks_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/masks/event/segmentation_masks'

# events_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/event/NIR_SWIR'
# notevents_path = '/home/cristopher/Desktop/TASOR-S2/dataset_creation/dataset/images/notevent/NIR_SWIR'

# masks_path = os.path.join(dataset_path,'masks','event','segmentation_masks')
# notevents_path = os.path.join(dataset_path,'images','notevent','NIR_SWIR')





def geo_splitted_notevents_creation(dataset_path: str,
                                    new_dataset_path: str,
                                    test_scenes: list,
                                    train_split_ratio: float = 0.8,
                                    val_split_ratio: float = 0.1,
                                    test_split_ratio: float = 0.1,
                                    seed: int = 42
                                    ):
    ### Dataset path corresponds to the original dataset of the images
    
    # Empty mask for the notevent patches
    mask = Image.fromarray(np.zeros((256,256,3),dtype=np.uint8))

    #Intitialice the seed
    random.seed(seed)
    
    masks_path = os.path.join(dataset_path,'masks','event','segmentation_masks')
    notevents_path = os.path.join(dataset_path,'images','notevent','NIR_SWIR')

    notevents_paths = os.listdir(notevents_path)
    random.shuffle(notevents_paths)



    
    #Obtain the number of different scenes of the dataset

    with open(os.path.join(dataset_path,'granules_completed.txt'),'r') as file:
        available_granules = file.readlines()

    scene_main_names_list = []
    for image_name in available_granules:
        
        scene_main_name = re.match(r'(.+)_[0-9]+_G',image_name).group(1)
        if re.match(r'(.+)_[0-9]+',scene_main_name): #This ensures tha scenes with multiple numbers such as Raung and Raung 1 are classified as the same
            scene_main_name = re.match(r'(.+)_[0-9]+',scene_main_name).group(1)
        
        if scene_main_name not in scene_main_names_list:

            scene_main_names_list.append(scene_main_name)
            # print(scene_main_name)

    n_scenes_train_val = (len(scene_main_names_list) - len(test_scenes)) #Substract the number of scenes selected for testing

    # Define maximum indexes and number of images per scene for training, validation and testing

    train_max_idx = int(train_split_ratio* len(os.listdir(masks_path)))
    val_max_idx = int(val_split_ratio* len(os.listdir(masks_path)))
    test_max_idx = int(test_split_ratio* len(os.listdir(masks_path)))

    max_n_train_images = int(train_max_idx/n_scenes_train_val)
    max_n_val_images = int(val_max_idx/n_scenes_train_val)
    max_n_test_images = int(test_max_idx/len(scenes_test_combination))


    #Initialice loop variables
    val_idx = 0
    test_idx = 0
    train_idx = 0
    train_scene_names_saved = []
    val_scene_names_saved = []
    test_scene_names_saved = []

    for image_name in notevents_paths:
        
        scene_main_name = re.match(r'(.+)_[0-9]+_G',image_name).group(1)
        if re.match(r'(.+)_[0-9]+',scene_main_name): #This ensures tha scenes with multiple numbers such as Raung and Raung 1 are classified as the same
            scene_main_name = re.match(r'(.+)_[0-9]+',scene_main_name).group(1)

        # # Extract the coordinates of the patch
        # numbers = re.findall(r'\((.*?)\)', image_name)
        # numbers_list = np.array([int(num) for num in numbers[0].split(',')])
        
        # if not (numbers_list==0).any(): 
        ###### NOT IMPLEMENTED YET THE CASE WHEN THE IMAGES ARE NOT IN THE BORDERS

        if train_idx >= train_max_idx and val_idx >= val_max_idx  and test_idx >= test_max_idx:
            break

        with open(os.path.join(notevents_path,image_name),'rb') as image_file:
            image = pickle.load(image_file)
            image_uint8 = np.round(np.multiply(image,255)).astype(np.uint8)
            image_img = Image.fromarray(image_uint8)

        ##### TESTING SPLIT #####

        if scene_main_name in scenes_test_combination:
            if test_scene_names_saved.count(scene_main_name) < max_n_test_images:
                test_scene_names_saved.append(scene_main_name)

                mask.save(os.path.join(os.path.join(new_dataset_path,'test','masks'),image_name.replace('_NIR_SWIR.pkl','_mask.png')))
                image_img.save(os.path.join(os.path.join(new_dataset_path,'test','images'),image_name.replace('.pkl','.png')))

            if test_idx >= test_max_idx:
                continue
            else:
                test_idx +=1

        else:

            ##### VALIDATION SPLIT #####
            
            if val_idx < val_max_idx:
                if val_scene_names_saved.count(scene_main_name) <= max_n_val_images:
                    val_scene_names_saved.append(scene_main_name)

                    mask.save(os.path.join(os.path.join(new_dataset_path,'val','masks'),image_name.replace('_NIR_SWIR.pkl','_mask.png')))
                    image_img.save(os.path.join(os.path.join(new_dataset_path,'val','images'),image_name.replace('.pkl','.png')))

                    
                    val_idx +=1
            else:

            ##### TRAINING SPLIT #####
                
                if train_scene_names_saved.count(scene_main_name) <= max_n_train_images:
                    train_scene_names_saved.append(scene_main_name)
                    
                    mask.save(os.path.join(os.path.join(new_dataset_path,'train','masks'),image_name.replace('_NIR_SWIR.pkl','_mask.png')))
                    image_img.save(os.path.join(os.path.join(new_dataset_path,'train','images'),image_name.replace('.pkl','.png')))
                    
                    train_idx +=1
    print(train_idx,val_idx,test_idx)
    print(train_max_idx,val_max_idx,test_max_idx)


# geo_splitted_notevents_creation(dataset_path = dataset_path,test_scenes = scenes_test_combination)

In [21]:
import os
import re
import pickle
import numpy as np


from PIL import Image
import random

import os
import re
import pickle
import numpy as np

from itertools import combinations
import random
from copy import deepcopy

import argparse

# new_dataset_path = os.path.dirname(os.path.basename(__file__),'train_dataset_test')
new_dataset_path = os.path.join(os.getcwd(),'train_dataset_test')
# os.makedirs(new_dataset_path,exist_ok=True)

create_dataset_folders(new_dataset_path)

dataset_path = os.path.join(os.path.dirname(os.getcwd()),'dataset_creation','dataset') #Provisional path

seed = 42
test_split_ratio = 0.1



event_scenes_dict = get_n_events_per_scene_dict(dataset_path=dataset_path,seed=seed)


# Maximum number of events in tests
n_events_test_max = int(test_split_ratio * sum(event_scenes_dict.values()))

scenes_test_combination = find_scenes_combination(input_dict=event_scenes_dict,n_events_max=n_events_test_max)

geo_splitted_events_creation(dataset_path=dataset_path,
                             new_dataset_path = new_dataset_path,
                             event_scenes_dict = event_scenes_dict,
                             scenes_test_combination = scenes_test_combination,seed=seed)

geo_splitted_notevents_creation(dataset_path = dataset_path,new_dataset_path=new_dataset_path,test_scenes = scenes_test_combination)


Events generated.
572 71 71
572 71 71
